In [ ]:
!git clone https://github.com/openai/CLIP.git

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 파일들을 로드하고 전처리
def load_images(image_folder):
    images = []
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('png', 'jpg', 'jpeg'))]

    for image_file in image_files:
        image = preprocess(Image.open(image_file)).unsqueeze(0).to(device)
        images.append((image_file, image))

    return images

# 이미지 임베딩 계산
def get_image_features(images):
    image_features = []
    for image_file, image in images:
        with torch.no_grad():
            image_feature = model.encode_image(image)
            image_features.append((image_file, image_feature))
    return image_features

# 텍스트 쿼리 임베딩 계산
def get_text_features(text_query):
    with torch.no_grad():
        text_tokens = clip.tokenize([text_query]).to(device)
        text_features = model.encode_text(text_tokens)
    return text_features

# 유사도 계산
def calculate_similarity(image_features, text_features):
    similarities = []
    for image_file, image_feature in image_features:
        similarity = torch.cosine_similarity(image_feature, text_features, dim=1).item()
        similarities.append((image_file, similarity))
    return similarities

# 이미지 검색 함수
def search_images(image_folder, text_query):
    global image_features
    images = load_images(image_folder)
    image_features = get_image_features(images)
    text_features = get_text_features(text_query)

    similarities = calculate_similarity(image_features, text_features)
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities

# 이미지 폴더와 텍스트 쿼리 설정
image_folder = '/content/image'
text_query = "강아지 캐릭터"

# 이미지 검색
results = search_images(image_folder, text_query)

# 결과 출력
for image_file, similarity in results:
    print(f"Image: {image_file}, Similarity: {similarity}")


In [ ]:
import torch
import clip
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 파일들을 로드하고 전처리
def load_images(image_folder):
    images = []
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('png', 'jpg', 'jpeg'))]

    for image_file in image_files:
        image = preprocess(Image.open(image_file)).unsqueeze(0).to(device)
        images.append((image_file, image))

    return images

# 이미지 임베딩 계산
def get_image_features(images):
    image_features = []
    for image_file, image in images:
        with torch.no_grad():
            image_feature = model.encode_image(image)
            image_features.append((image_file, image_feature))
    return image_features

# 해시태그 임베딩 계산
def get_hashtag_features(hashtags):
    with torch.no_grad():
        text_tokens = clip.tokenize(hashtags).to(device)
        text_features = model.encode_text(text_tokens)
    return text_features

# 유사도 계산
def calculate_similarity(image_features, text_features):
    similarities = []
    for image_file, image_feature in image_features:
        similarity = torch.cosine_similarity(image_feature, text_features, dim=1)
        similarities.append((image_file, similarity))
    return similarities

# 해시태그 생성 함수
def generate_hashtags(image_folder, hashtags):
    images = load_images(image_folder)
    image_features = get_image_features(images)
    hashtag_features = get_hashtag_features(hashtags)

    similarities = calculate_similarity(image_features, hashtag_features)

    image_hashtags = {}
    for image_file, similarity in similarities:
        # 가장 유사한 해시태그 5개 선택
        top_indices = similarity.topk(3).indices.cpu().numpy()
        image_hashtags[image_file] = [hashtags[i] for i in top_indices]

    return image_hashtags

# 이미지 폴더와 해시태그 리스트 설정
image_folder = './'  # 여기에 이미지 폴더 경로를 입력하세요
# hashtags = [
#     "#nature", "#travel", "#food", "#art", "#technology",
#     "#fashion", "#sports", "#music", "#animals", "#people"
# ]
hashtags = [
    "#character", "#people", "#travel", "#technology", "#art",
    "#animals", "#computer", "#fashion", "#game", "#mobile"
]
# hashtags = [
#     "#캐릭터","#사람","#여행","#기술","#예술","#동물","#컴퓨터","#패션", "#게임", "#모바일"
# ]

# 해시태그 생성
image_hashtags = generate_hashtags(image_folder, hashtags)

# 결과 출력
for image_file, tags in image_hashtags.items():
    print(f"Image: {image_file}, Hashtags: {tags}")


#### EfficientNet + faiss (0.6)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image, UnidentifiedImageError
import numpy as np
import os
import faiss
import matplotlib.pyplot as plt

# EfficientNet 모델 로드 및 임베딩 벡터 추출 레이어 설정
base_model = models.efficientnet_b0(pretrained=True)
base_model.classifier = nn.Identity()  # Remove the classification layer

# 이미지 전처리 함수
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_image_embedding(image_path, model, device):
    try:
        # 이미지 로드 및 전처리
        img = Image.open(image_path).convert('RGB')
        img_tensor = preprocess(img).unsqueeze(0).to(device)

        # 임베딩 벡터 생성
        model.eval()
        with torch.no_grad():
            embedding_vector = model(img_tensor).cpu().numpy().flatten()
        return embedding_vector
    except UnidentifiedImageError:
        print(f"Cannot identify image file {image_path}. Skipping.")
        return None

def get_embeddings_from_folder(folder_path, model, device):
    embeddings = []
    image_paths = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('jpg', 'jpeg', 'png')):
            image_path = os.path.join(folder_path, file_name)
            embedding = get_image_embedding(image_path, model, device)
            if embedding is not None:
                embeddings.append(embedding)
                image_paths.append(image_path)
    return np.array(embeddings), image_paths

def plot_image_groups(groups):
    for idx, group in enumerate(groups):
        print("=================group {}=================".format(idx))
        plt.figure(figsize=(20, 20))
        for i, image_path in enumerate(group):
            plt.subplot(1, len(group), i + 1)
            img = Image.open(image_path)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

# 예시 폴더 경로
folder_path = "pickle/"

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

# 폴더 내 모든 이미지에 대한 임베딩 벡터 생성
embeddings, image_paths = get_embeddings_from_folder(folder_path, base_model, device)

# FAISS를 이용한 코사인 유사도 측정
index = faiss.IndexFlatIP(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

D, I = index.search(embeddings, k=len(embeddings))  # 모든 이미지에 대해 유사도 측정

# 유사도 0.6 이상인 그룹 생성
threshold = 0.6
groups = []
visited = set()

for i in range(len(embeddings)):
    if i in visited:
        continue
    group = [image_paths[i]]
    visited.add(i)
    for j in range(1, len(I[i])):
        if D[i][j] >= threshold and I[i][j] not in visited:
            group.append(image_paths[I[i][j]])
            visited.add(I[i][j])
    groups.append(group)

# 결과 출력
plot_image_groups(groups)


C:\Users\ben81\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ben81\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Cannot identify image file pickle/IMG_20231017_201401_429.jpg. Skipping.
Cannot identify image file pickle/IMG_20240111_162248_875.jpg. Skipping.
Cannot identify image file pickle/IMG_20240111_162320_969.jpg. Skipping.
Cannot identify image file pickle/IMG_20240124_210435_771.jpg. Skipping.
Cannot identify image file pickle/IMG_20240214_224130_331.jpg. Skipping.
Cannot identify image file pickle/IMG_20240214_231024_645.jpg. Skipping.
Cannot identify image file pickle/IMG_20240404_234805_048.jpg. Skipping.
Cannot identify image file pickle/IMG_20240411_201256_965.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_140836_666.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_560.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_642.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_721.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_757.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_2050

#### CLIP + faiss (0.8)

In [ ]:
import torch
import torch.nn as nn
from PIL import Image, UnidentifiedImageError
import numpy as np
import os
import faiss
import matplotlib.pyplot as plt
import clip

# CLIP 모델 및 전처리기 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

def get_image_embedding(image_path, model, preprocess, device):
    try:
        # 이미지 로드 및 전처리
        img = Image.open(image_path).convert('RGB')
        img_tensor = preprocess(img).unsqueeze(0).to(device)

        # 임베딩 벡터 생성
        with torch.no_grad():
            embedding_vector = model.encode_image(img_tensor).cpu().numpy().flatten()
        return embedding_vector
    except UnidentifiedImageError:
        print(f"Cannot identify image file {image_path}. Skipping.")
        return None

def get_embeddings_from_folder(folder_path, model, preprocess, device):
    embeddings = []
    image_paths = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('jpg', 'jpeg', 'png')):
            image_path = os.path.join(folder_path, file_name)
            embedding = get_image_embedding(image_path, model, preprocess, device)
            if embedding is not None:
                embeddings.append(embedding)
                image_paths.append(image_path)
    return np.array(embeddings, dtype=np.float32), image_paths

def plot_image_groups(groups):
    for idx, group in enumerate(groups):
        print("=================group {}=================".format(idx))
        plt.figure(figsize=(20, 20))
        for i, image_path in enumerate(group):
            plt.subplot(1, len(group), i + 1)
            img = Image.open(image_path)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

# 예시 폴더 경로
folder_path = "pickle/"

# 폴더 내 모든 이미지에 대한 임베딩 벡터 생성
embeddings, image_paths = get_embeddings_from_folder(folder_path, model, preprocess, device)

# FAISS를 이용한 코사인 유사도 측정
index = faiss.IndexFlatIP(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

D, I = index.search(embeddings, k=len(embeddings))  # 모든 이미지에 대해 유사도 측정

threshold = 0.8
groups = []
visited = set()

for i in range(len(embeddings)):
    if i in visited:
        continue
    group = [image_paths[i]]
    visited.add(i)
    for j in range(1, len(I[i])):
        if D[i][j] >= threshold and I[i][j] not in visited:
            group.append(image_paths[I[i][j]])
            visited.add(I[i][j])
    groups.append(group)

# 결과 출력
plot_image_groups(groups)


C:\Users\ben81\anaconda3\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Cannot identify image file pickle/IMG_20231017_201401_429.jpg. Skipping.
Cannot identify image file pickle/IMG_20240111_162248_875.jpg. Skipping.
Cannot identify image file pickle/IMG_20240111_162320_969.jpg. Skipping.
Cannot identify image file pickle/IMG_20240124_210435_771.jpg. Skipping.
Cannot identify image file pickle/IMG_20240214_224130_331.jpg. Skipping.
Cannot identify image file pickle/IMG_20240214_231024_645.jpg. Skipping.
Cannot identify image file pickle/IMG_20240404_234805_048.jpg. Skipping.
Cannot identify image file pickle/IMG_20240411_201256_965.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_140836_666.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_560.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_642.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_721.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_205032_757.jpg. Skipping.
Cannot identify image file pickle/IMG_20240413_2050

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some random data
x = np.linspace(0, 10, 100)
y = np.sin(x)

# Create the plot
plt.plot(x, y)
plt.title('Sine Wave')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)

# Display the plot
plt.show()